In [24]:
#-------------------------------------------------------------------#
##### GIS + Education + Real Estate Market Project #####
##### Author : Shin Hyunjin (KAIST Business School, SK Securities)
#-------------------------------------------------------------------#

In [1]:
import folium
import pandas as pd
import xlwings as xw
import json

In [2]:
#Basic Setting
seoul_map = folium.Map(location = [37.561437, 126.9751701])

In [3]:
seoul_map = folium.Map(location = [37.561437, 126.9751701], popup = '서울시청').add_to(seoul_map)

In [4]:
seoul_map

In [5]:
seoul_map = folium.Map(location = [37.561437, 126.9751701], zoom_start = 14)

In [6]:

#load data
school_path = r'c:/users/shinhyunjin/dropbox/data/KB Real Estate Data/2019school.xlsx'
school_raw = pd.read_excel(school_path)
school_data = school_raw[['정보공시 \n 학교코드', '학교명', '위도', '경도' ]]
school_data.columns = ['학교코드', '학교명', '위도', '경도']
school_df = school_data.drop_duplicates()


In [7]:
# function for data cleaning

def graduate_preprocessing(path):
    
    raw_graduate = pd.read_excel(path, sheet_name = '2019_졸업생의 진로 현황(중)')
    
    select_col = raw_graduate[['지역', '학교명', '정보공시 \n 학교코드', '졸업자.2',
                           '(특수목적고)과학고 진학자.2','(특수목적고)외고ㆍ국제고 진학자.2']]
    select_col.columns = ['지역', '학교명' ,'학교코드', '졸업자', '과고', '외고']
    graduate_data = select_col.drop(0)
    graduate_data['과고'] = pd.to_numeric(graduate_data['과고'])
    graduate_data['외고'] = pd.to_numeric(graduate_data['외고'])
    graduate_data['졸업자'] = pd.to_numeric(graduate_data['졸업자'])
    graduate_data['총합'] = graduate_data['과고']+graduate_data['외고']

    def get_sido(x):
    
        temp = x.split(' ')[0]
        if len(temp) != 4:
             return temp[:2]
        else:
             return temp[0] + temp[2]
    
    
    graduate_data['시도'] = graduate_data['지역'].dropna().apply(get_sido)
    graduate_data['구군'] = graduate_data['지역'].dropna().apply(lambda x: x.split(' ')[1])

    graduate_data.at[588, '시도'] = '부산'
    graduate_data.at[588, '구군'] = '기장군'
    graduate_data.at[3011,'시도'] = '경북'
    graduate_data.at[3011,'구군'] = '예천군'
    
    return graduate_data


In [8]:
## shchool data


graduate_path = r'c:/users/shinhyunjin/dropbox/data/KB Real Estate Data/2019_edu_result.xlsx'
gradu_df = graduate_preprocessing(graduate_path)

total_school_df = pd.merge(gradu_df, school_df, how = 'inner', right_on = '학교코드',
                           left_on = '학교코드')

In [9]:
# data cleaning 2
seoul_school = total_school_df[total_school_df['시도'] == '서울']
good_school = seoul_school[seoul_school['총합'] > 0]
seoul_map = folium.Map(location = [37.5614378, 126.9751701], zoom_start =11)

In [10]:
# loop for good middle school
for n in good_school.index:
    folium.Marker([good_school['위도'][n], good_school['경도'][n]],
                  popup = good_school['학교명_x'][n]).add_to(seoul_map)
    

In [11]:
seoul_school['비율'] = seoul_school['총합'] / seoul_school['졸업자']*100
good_school = seoul_school[seoul_school['비율'] >=3]
bad_school = seoul_school[seoul_school['비율'] < 3]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
seoul_map = folium.Map(location = [37.5614378, 126.9751701], zoom_start = 11)
seoul_map

In [13]:
# 특목고

for n in good_school.index:
    folium.CircleMarker([good_school['위도'][n], good_school['경도'][n]],
                          color = 'crimson', fill_color = 'crimson', radius = 7).add_to(seoul_map)
seoul_map

In [14]:
# shit school
for n in bad_school.index:
    folium.CircleMarker([bad_school['위도'][n], bad_school['경도'][n]],
                        color = '#3186cc', fill_color = '#3186cc', radius = 7).add_to(seoul_map)
                        
seoul_map

In [15]:
# exmample : local area

sido_school = total_school_df[total_school_df['시도'] == '대전']
sido_school['비율'] = sido_school['총합'] / sido_school['졸업자'] * 100
good_school = sido_school[sido_school['비율'] >=3]
bad_school = sido_school[sido_school['비율'] <3]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [16]:
#### 지방 : 특목고 + 일반고 GIS Data ####

sido_map = folium.Map(location = [36.350461, 127.38263], zoom_start =11)

for n in good_school.index:
    
    folium.CircleMarker([good_school['위도'][n], good_school['경도'][n]],
                       color = 'crimson', fill_color = 'crimson', radius = 7).add_to(sido_map)
    
for n in bad_school.index:
    
    folium.CircleMarker([bad_school['위도'][n], bad_school['경도'][n]],
                       color = '#3186cc', fill_color = '#3186cc', radius = 7).add_to(sido_map)
    
sido_map

In [17]:
# -----------------------------------------------------------------------------#
# Data Cleaning for local adress code data#

local_code = pd.read_excel(r'c:/users/shinhyunjin/dropbox/data/KB Real Estate Data/GIS data/KIKcd_B.20190701.xlsx')
                           
local_code['법정동코드'] = local_code['법정동코드'].apply(lambda x: str(x)[:5])
filtered_code = local_code[['법정동코드','시도명', '시군구명']].drop_duplicates()
filtered_code.dropna(inplace = True)
filtered_code.loc[20477] = ('36110', '세종', '세종')

In [18]:
# function for data cleaning

def get_sido(x):
    
    temp = x.split(' ')[0]
    
    if len(temp) != 4:
        return temp[:2]
    else:
        return temp[0]+temp[2]
    
filtered_code['시도명'] = filtered_code['시도명'].dropna().apply(get_sido)
filtered_code['시군구명'] = filtered_code['시군구명'].dropna().apply(lambda x: x.split(' ')[-1])


In [19]:

def KBpriceindex_preprocessing(path, data_type):
    wb = xw.Book(path)
    sheet = wb.sheets[data_type]
    row_num = sheet.range(1,1).end('down').end('down').end('down').row
    data_range = 'A2:GE' + str(row_num)
    raw_data = sheet[data_range].options(pd.DataFrame, index = False, header =True).value

    bignames = '서울 대구 부산 대전 광주 인천 울산 세종 경기 강원 충북 충남 전북 전남 경북 경남 제주도 6개광역시 5개광역시 수도권 기타지방 구분 전국'
    bigname_list = bignames.split(' ')
  
    big_col = list(raw_data.columns)
    small_col = list(raw_data.iloc[0])

#column cleaning
    for num, gu_data in enumerate(small_col): # order saved
        if gu_data == None:
            small_col[num] = big_col[num]
        
        check = num
    
        while True:
            if big_col[check] in bigname_list:
                big_col[num] = big_col[check]
                break
            else:
                check = check-1
            
    big_col[129] = '경기'
    big_col[130] = '경기'
    small_col[185] = '서귀포'

    raw_data.columns = [big_col, small_col]
    new_col_data = raw_data.drop([0,1])

# date list #

    index_list = list(new_col_data['구분']['구분'])
    new_index =[]

    for num, raw_index in enumerate(index_list):
        
        temp = str(raw_index).split('.')
        if int(temp[0]) > 12:
            if len(temp[0]) ==2:
                new_index.append('19'+temp[0] + '.' + temp[1])
            else:
                new_index.append(temp[0] + '.' + temp[1])
        else:
             
             new_index.append(new_index[num-1].split('.')[0]+'.'+temp[0])

    new_col_data.set_index(pd.to_datetime(new_index), inplace = True)
    cleaned_data = new_col_data.drop(('구분', '구분'), axis = 1)
    return cleaned_data

In [23]:
# Real Estate Price Data #
path = r'c:/users/shinhyunjin/dropbox/data/KB Real Estate Data/1906data.xlsx'
price_index = KBpriceindex_preprocessing(path, '매매apt')
diff_pct = ((price_index.loc['2019-6-1']/price_index.loc['2018-6-1'])-1)*100
diff_df = pd.DataFrame({'증감률':diff_pct})

In [24]:
# Merge Data #

filtered_code.index = [filtered_code['시도명'], filtered_code['시군구명']]

code = []

for local in diff_df.index:
    
    if local[0] in filtered_code.index:
        
        temp_df = filtered_code.loc[local[0]]
        
        if local[1] in temp_df.index:
            code.append(temp_df.loc[local[1]]['법정동코드']) # upper - downward indexing
        elif local[1] + '시' in temp_df.index:
            
            code.append(temp_df.loc[local[1]+'시']['법정동코드'])
        elif local[1] == '세종':
            code.append('36110')
        else:
            code.append('')
    else:
        code.append('')
diff_df['법정동코드'] = code
diff_df

증감률  법정동코드
전국    전국       1.13775       
서울    서울       7.48669       
      강북       7.93094       
      강북구      8.88566  11305
      광진구      5.21362  11215
      노원구      9.04136  11350
      도봉구      9.55686  11320
      동대문구     8.10834  11230
      마포구      8.19572  11440
      서대문구     8.11438  11410
      성동구      6.35925  11200
      성북구      7.64011  11290
      용산구      7.89074  11170
      은평구      7.80029  11380
      종로구      4.95606  11110
      중구       5.51697  11140
      중랑구      6.08363  11260
      강남       7.10565       
      강남구       3.2973  11680
      강동구      5.42455  11740
      강서구      8.27748  11500
      관악구       9.1876  11620
      구로구      10.4724  11530
      금천구      7.68976  11545
      동작구        10.06  11590
      서초구      5.70316  11650
      송파구      4.37277  11710
      양천구        10.05  11470
      영등포구     10.3185  11560
6개광역시 6개광역시 -0.0198013       
...                ...    ...
전북    익산       -0.4379  45140
      군산       -4.3546  45130
전남    전남       1.34528       
      목포     -0.566762  46110
      순천       1.81094  46150
      광양       2.03435  46230
      여수       2.33887  46130
경북    경북      -6.09231       
      포항      -6.97267  47110
      남구      -5.14233  47111
      북구        -8.046  47113
      구미      -7.23087  47190
      경산      -3.00777  47290
      안동      -5.87358  47170
      김천           NaN  47150
경남    경남      -5.45464       
      창원      -4.73361  48120
      마산합포구   -4.77356  48125
      마산회원구   -3.93067  48127
      성산구     -4.87147  48123
      의창구     -6.18982  48121
      진해구     -3.92877  48129
      양산      -2.23349  48330
      거제      -7.11904  48310
      진주      -5.89563  48170
      김해      -8.43578  48250
      통영      -4.61072  48220
제주도   제주도          NaN       
      서귀포    -0.689734       
기타지방  기타지방    -4.07586       

[186 rows x 2 columns]

In [28]:
# Visualization #

rfile = open(r'c:/users/shinhyunjin/dropbox/data/KB Real Estate Data/GIS Data/TL_SCCO_SIG_WGS84.json', 'r', encoding= 'utf-8').read()
jsonData = json.loads(rfile)

In [29]:
korea_map= folium.Map(location = [36,127], zoom_start = 7)

In [30]:
korea_map.choropleth(
geo_data = jsonData,
data = diff_df,
columns = ['법정동코드', '증감률'],
    key_on = 'feature.properties.SIG_CD',
    fill_color = 'RdYlGn',
    fill_opacity = 0.7,
    line_opacity = 0.5,
    legend_name = '증감률(%)')

C:\ProgramData\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [31]:
korea_map.save('c:/users/shinhyunjin/dropbox/data/KB Real Estate Data/부동산 증감 지도.html')